# Information Retrieval project 
**Authors:** Arduini L., Menchini L., Namaki Ghaneh D., Petruzzella C.

# 1. Preprocessing

In [19]:
!pip install ir_datasets
!pip install nltk


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 151.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 226.7 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.7/284.7 kB 254.3 kB/s eta 0:00:00a 0:00:01
Using cached click-8.1.7-py3-none-any.whl (97 kB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [20]:
import ir_datasets

# Load the MS MARCO dataset
dataset = ir_datasets.load("msmarco-passage")

In [21]:
# print the first document in the dataset

import random

# Initialize a flag to check if a document has been printed
document_printed = False

# Iterate over the documents in the dataset
for doc in dataset.docs_iter():
    if not document_printed:
        if random.random() < 0.01:  # Adjust the probability as needed
            print(doc.text)
            document_printed = True
            break

Common Treatments done by Medical Tourists in Costa Rica. Known initially for its excellent dental surgery services, medical tourism in Costa Rica has spread to a variety of other medical procedures, including: General and cosmetic dentistry; Cosmetic surgery; Aesthetic procedures (botox, skin resurfacing etc) Bariatric and Laparoscopic surgery


In [23]:
import re
import string
import nltk

nltk.download("stopwords", quiet=True)

def preprocess(s):
    # lowercasing
    s = s.lower()
    # ampersand
    s = s.replace("&", " and ")
    # special chars
    s = s.translate(dict([(ord(x), ord(y)) for x, y in zip("‘’´“”–-", "'''\"\"--")]))
    # acronyms
    s = re.sub(r"\.(?!(\S[^. ])|\d)", "", s)
    # remove punctuation
    s = s.translate(str.maketrans(string.punctuation, " " * len(string.punctuation)))
    # strip whitespaces
    s = s.strip()
    while "  " in s:
        s = s.replace("  ", " ")
    # tokeniser
    s = s.split()
    # stopwords
    stopwords = nltk.corpus.stopwords.words('english')
    s = [t for t in s if t not in stopwords]
    # stemming
    stemmer = nltk.stem.PorterStemmer().stem
    s = [stemmer(t) for t in s]
    return s

In [24]:
import time

def profile(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        ms = (end - start) * 1000
        print(f"{f.__name__} ({ms:.3f} ms)")
        return result
    return f_timer

In [25]:
from collections import Counter
from tqdm.auto import tqdm

@profile
def build_index(dataset):
    lexicon = {}
    doc_index = []
    inv_d, inv_f = {}, {}
    termid = 0

    num_docs = 0
    total_dl = 0
    total_toks = 0
    for docid, doc in tqdm(enumerate(dataset.docs_iter()), desc='Indexing', total=dataset.docs_count()):
        tokens = preprocess(doc.text)
        token_tf = Counter(tokens)
        for token, tf in token_tf.items():
            if token not in lexicon:
                lexicon[token] = [termid, 0, 0]
                inv_d[termid], inv_f[termid] =  [], []
                termid += 1
            token_id = lexicon[token][0]
            inv_d[token_id].append(docid)
            inv_f[token_id].append(tf)
            lexicon[token][1] += 1
            lexicon[token][2] += tf
        doclen = len(tokens)
        doc_index.append((str(doc.doc_id), doclen))
        total_dl += doclen
        num_docs += 1


    stats = {
        'num_docs': 1 + docid,
        'num_terms': len(lexicon),
        'num_tokens': total_dl,
    }
    return lexicon, {'docids': inv_d, 'freqs': inv_f}, doc_index, stats

/Users/danielnamaki/Documents/University/ir-project/mircv_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
lex, inv, doc, stats = build_index(dataset)

Indexing:  20%|██        | 1806321/8841823 [15:02<58:35, 2001.43it/s]  


KeyboardInterrupt: 

# 2. Indexing

In [ ]:
import math

class InvertedIndex:

    class PostingListIterator:
        def __init__(self, docids, freqs, doc):
            self.docids = docids
            self.freqs = freqs
            self.pos = 0
            self.doc = doc

        def docid(self):
            if self.is_end_list():
                return math.inf
            return self.docids[self.pos]

        def score(self):
            if self.is_end_list():
                return math.inf
            return self.freqs[self.pos]/self.doc[self.docid()][1]

        def next(self, target = None):
            if not target:
                if not self.is_end_list():
                    self.pos += 1
            else:
                if target > self.docid():
                    try:
                        self.pos = self.docids.index(target, self.pos)
                    except ValueError:
                        self.pos = len(self.docids)

        def is_end_list(self):
            return self.pos == len(self.docids)


        def len(self):
            return len(self.docids)


    def __init__(self, lex, inv, doc, stats):
        self.lexicon = lex
        self.inv = inv
        self.doc = doc
        self.stats = stats

    def num_docs(self):
        return self.stats['num_docs']

    def get_posting(self, termid):
        return InvertedIndex.PostingListIterator(self.inv['docids'][termid], self.inv['freqs'][termid], self.doc)

    def get_termids(self, tokens):
        return [self.lexicon[token][0] for token in tokens if token in self.lexicon]

    def get_postings(self, termids):
        return [self.get_posting(termid) for termid in termids]

In [ ]:
inv_index = InvertedIndex(lex, inv, doc, stats)

Compress and save the index components

In [ ]:
import gzip
import pickle

with gzip.open('lexicon.pickle.gz', 'wb') as f:
  pickle.dump(lex, f)
with gzip.open('inverted_file.pickle.gz', 'wb') as f:
  pickle.dump(inv, f)
with gzip.open('document_index.pickle.gz', 'wb') as f:
  pickle.dump(doc, f)
with gzip.open('stats.pickle.gz', 'wb') as f:
  pickle.dump(stats, f)

# 3. Query processing

# 4. Evaluation